In [1]:
import re
from faker import Faker
import spacy
import random
from spacy.tokens import DocBin

fake = Faker()

In [2]:
templates = [
    "Patient: Ling Chi, Gender: Male, Admit Date: 10/02/22, VITALS: TEMP. {VITAL-TEMP}, PULSE: {VITAL-PULSE}, RESP: {VITAL-RESP}",
    "Name: John Brown Account No: 12345 Attending Physician: JeffT. Moore, M.D. Consulting Physician Adm Date: 11/12/1999 DOB: 09/10/44 Community General Hospital Anytown, USA PATIENT ADMISSION ASSESSMENT Date 1/IN44 Time 1600 Admit from: Dr. Office ERSHome Other Health Care Facility:. Reason for Adrission: (Patient's own words). Hechia Surgry Primary Care Physician:. 1i Mloore Previous Health Problems (Patient Only) NA 1. Diabetes 2. Epilepsy/seizure disorder 3. High Blood Pressure 4. Heart Disease 5. Kidney Disease 6. Cancer Medications Anesthetics Other. Comments by nymber and other hosptalationsurgenes Dreyibus lnia repr 7. Respiratory 8. GU/GYN 9. GI 10. Steroid use D 11. Flu Vaccine Current CP N 12. Pneumonia Vaccine current Y(Nnog give patient information VITALS: TEMP. {VITAL-TEMP} PULSE {VITAL-PULSE} RESP {VITAL-RESP} B/P {VITAL-BP} mmHg Wt 19096s ALLERGIES HABITS NVA Food Environmental Dyes Ruber/atexbalons: Yes (NONA Yrs. Other Tobacco: Yes_No per day. Alcohol: Yes-No) per day. Yrs. Chew: Yes No2 yrs. Drug use/abuse: Yes Noype. per day. MEDICATIONS: Brought to hospital: YN Sent Home: Y N To Pharmacy: Y N Personal Pharmacy. NAME NA DOSE/FREQUENCY TIME OF LAST DOSE Rx OTC Do you use herbs or other alternative medications: Yes ONo List: Lainko OKAcSIonly Orientation to room: -3 Calllight Pelephone bathroom location_tghts meal time visitor polcy Personal belongings kept on person or at bedside: None Eyeglasses Contacts Hearing aids Glass eye Walker Wheelchair WE ARE NOT RESPONSIBLE FORI BELONGINGS/NALUABLES. WE STRONGLY SUGGEST YOU SEND ITEMS HOME. lacknowledge that the above belongings are in. my possession. Ihave received information on Advanced Directives and Organ Tissue Denture (Upper/Lower) Partial (Upper/Lower) Crutches Clothing/Other Money $. Donation. (circle) Home Safe Kept Jewelry. (circle) Home Safe Kept Patient gDe Witness hc",
    "Patient: Mary Jane, ID: 99124, DOB: 02/05/86, Admit Date: 12/2/19, Hospital: StarChrist Hosptial for Cancer Patients, VITALS: TEMP. {VITAL-TEMP}, PULSE: {VITAL-PULSE}",
    "Patient ID: 595357, Name: Chris Doe, DOB: 4/4/44, VITALS: RESP {VITAL-RESP}/min, B/P {VITAL-BP}mmHg",
    "Name: Karen Singh, Gender: Other, Admit Date: 10/24/2024, VITALS: B/P {VITAL-BP}, TEMP. {VITAL-TEMP}, RESP: {VITAL-RESP}",
    "Name: Issac Ferrari, ID: 22928, Hospital: Greenbull Hosptial, VITALS: TEMP. {VITAL-TEMP}, PULSE: {VITAL-PULSE}, B/P {VITAL-BP}",
    "Patient: Lionel Ronaldo, Admit Date: 09/11/1991, Hospital: Madrid Hospital for Children, VITALS: RESP {VITAL-RESP}, TEMP. {VITAL-TEMP}",
    "Name: Sam Wood, DOB: 01/01/99, Admit Date: 2/27/20, VITALS: TEMP. {VITAL-TEMP}, HR: {VITAL-PULSE}",
    "Patient ID: 22127, Name: Ling Diren, Hospital: Apollo Hospital, VITALS: B/P {VITAL-BP}mmHg, TEMP. {VITAL-TEMP}F",
    "Patient: Maria Gates, Gender: F, Admit Date: 07/22/2021, VITALS: TEMP. {VITAL-TEMP}, RESP: {VITAL-RESP}",
    "Name: Karan Gupta, ID: 792424, DOB: 7/9/24, VITALS: TEMP. {VITAL-TEMP}, HEART RATE: {VITAL-PULSE}, B/P {VITAL-BP}",
    "VITALS: RR {VITAL-RESP}, TEMP. {VITAL-TEMP}",
    "Name: Levy Gotham, TEMP. {VITAL-TEMP}F, PULSE: {VITAL-PULSE}bpm, B/P {VITAL-BP}mmHg",
    "Vital Readings: T - {VITAL-TEMP}°F, HR - {VITAL-PULSE}bpm, BP - {VITAL-BP}, RR - {VITAL-RESP}/min"
    "VITALS: Temp - {VITAL-TEMP}, Pulse - {VITAL-PULSE}, Resp - {VITAL-RESP}, BP - {VITAL-BP}",
    "Vitals - Temp: {VITAL-TEMP}, Pulse: {VITAL-PULSE}, Resp: {VITAL-RESP}, BP: {VITAL-BP}",
    "Vitals for Patient Thors Odinson: Temp - {VITAL-TEMP}, Pulse - {VITAL-PULSE}, BP - {VITAL-BP}",
    "Temp - {VITAL-TEMP}, Pulse - {VITAL-PULSE}, Resp - {VITAL-RESP}",
    "{VITAL-TEMP}F, {VITAL-PULSE}bpm, {VITAL-RESP}/min, BP: {VITAL-BP}mmHg",
    "Vitals - {VITAL-TEMP}°F, {VITAL-PULSE}bpm, {VITAL-RESP}/min, BP: {VITAL-BP}",
    "{VITAL-TEMP}°F, {VITAL-PULSE} bpm, {VITAL-RESP} breaths/min, BP: {VITAL-BP}",
    "Patient's Vitals: Temp - {VITAL-TEMP}, Pulse - {VITAL-PULSE}, Resp - {VITAL-RESP}, BP - {VITAL-BP} | Patient: Lisa Banks",
    "Patient: Orsborn Shin | Vitals - Resp: {VITAL-RESP}, Pulse: {VITAL-PULSE}, Temp: {VITAL-TEMP}, BP: {VITAL-BP}",
    "Vitals: BP - {VITAL-BP}, Temp - {VITAL-TEMP}, Resp - {VITAL-RESP}, Pulse - {VITAL-PULSE}",
    "Patient's Vitals: Pulse - {VITAL-PULSE} bpm, BP - {VITAL-BP} mm/Hg, Resp - {VITAL-RESP} /minutes, Temp - {VITAL-TEMP} Fahrenheit| Patient: Caroline Mendes",
    "Dear Doctor, please find below the latest vitals for patient: Temperature - {VITAL-TEMP}°F, Pulse - {VITAL-PULSE}beats/min, Respiratory Rate - {VITAL-RESP}/min, Blood Pressure - {VITAL-BP} mm/Hg. Kindly review and advise accordingly. Thank you.",
    "Dear Dr. Priya Khan, please find below the latest vitals for patient Aakash Gupta: - Temperature: {VITAL-TEMP} - Blood Pressure: {VITAL-BP} - Respiratory Rate: {VITAL-RESP} - Pulse Rate: {VITAL-PULSE}. These readings were taken at 15:03 on 09/27. Your attention to this matter is appreciated. Thank you.",
    "Abnormal Vitals Recorded: Temp - {VITAL-TEMP}°F (High), Pulse - {VITAL-PULSE}bpm (Low), BP - {VITAL-BP} (High), Resp - {VITAL-RESP}/min (Low)",
    "Vital Signs: HR - {VITAL-PULSE} bpm, BP - {VITAL-BP}, RR - {VITAL-RESP}/min, Temp - {VITAL-TEMP}°F",
    "T - {VITAL-TEMP}, HR - {VITAL-PULSE}, BP - {VITAL-BP}mmHg, RR - {VITAL-RESP}/min",
]

In [3]:
from faker import Faker

fake = Faker()


def generate_temperature():
    temperature = fake.pyfloat(right_digits=1, positive=True, min_value=80.0, max_value=105.5)
    return temperature


def generate_resp_rate():
    resp_rate = fake.random_int(min=8, max=29)
    return resp_rate


def generate_blood_pressure():
    bp_up = fake.random_int(min=80, max=150)
    bp_down = fake.random_int(min=60, max=90)
    bp = "{}/{}".format(bp_up, bp_down)
    return bp

def generate_pulse_rate():
    pulse = fake.random_int(min=40, max=140)
    return pulse

In [ ]:
#print(generate_blood_pressure())
#print(generate_pulse_rate())
#print(generate_resp_rate())
#print(generate_temperature())

In [14]:
placeholder_to_generator = {
    "VITAL-TEMP": generate_temperature,
    "VITAL-PULSE": generate_pulse_rate,
    "VITAL-RESP": generate_resp_rate,
    "VITAL-BP": generate_blood_pressure
}

In [24]:
def replace_placeholders(template, placeholder_to_generator):
    updated_text = template
    entities = []

    # Iterate over each placeholder in the template text
    while "{" in updated_text:
        # Find the next placeholder in the updated text
        match = re.search(r"\{([^{}]*)\}", updated_text)
        if not match:
            break
        placeholder = match.group(1)

        # Replace the placeholder with the generated value
        fake_value = placeholder_to_generator[placeholder]()
        start, end = match.span()
        updated_text = updated_text[:start] + str(fake_value) + updated_text[end:]

        # # Update the start and end positions for entities
        # print("Placeholder:", placeholder)
        # print("Start:", start)
        # print("End:", start + len(fake_value))
        entities.append((start, start + len(str(fake_value)), placeholder))

    return updated_text, {"entities": entities}

In [29]:
#Sample run of all templates one time

for template in templates:
    print(replace_placeholders(template, placeholder_to_generator))

('Patient: Ling Chi, Gender: Male, Admit Date: 10/02/22, VITALS: TEMP. 102.6, PULSE: 75, RESP: 8', {'entities': [(69, 74, 'VITAL-TEMP'), (83, 85, 'VITAL-PULSE'), (93, 94, 'VITAL-RESP')]})
("Name: John Brown Account No: 12345 Attending Physician: JeffT. Moore, M.D. Consulting Physician Adm Date: 11/12/1999 DOB: 09/10/44 Community General Hospital Anytown, USA PATIENT ADMISSION ASSESSMENT Date 1/IN44 Time 1600 Admit from: Dr. Office ERSHome Other Health Care Facility:. Reason for Adrission: (Patient's own words). Hechia Surgry Primary Care Physician:. 1i Mloore Previous Health Problems (Patient Only) NA 1. Diabetes 2. Epilepsy/seizure disorder 3. High Blood Pressure 4. Heart Disease 5. Kidney Disease 6. Cancer Medications Anesthetics Other. Comments by nymber and other hosptalationsurgenes Dreyibus lnia repr 7. Respiratory 8. GU/GYN 9. GI 10. Steroid use D 11. Flu Vaccine Current CP N 12. Pneumonia Vaccine current Y(Nnog give patient information VITALS: TEMP. 81.3 PULSE 80 RESP 14 B/P 84

In [48]:
#Add to json for easy readbility of generated data

import json

# Generate data for each template
data = [replace_placeholders(template, placeholder_to_generator) for template in templates]

# Write data to a JSON file
with open("C:\\Users\\HShah\\OneDrive - Genzeon Corporation\\Desktop\\ner_dataset\\vitals\\generated_data.json", "w") as file:
    json.dump(data, file, indent=4)


In [51]:
#Highlighting text to see if spans are correct

import json

def highlight_text(text, entities):
    highlighted_text = ""
    current_pos = 0

    for entity in entities:
        start, end, label = entity
        highlighted_text += text[current_pos:start]  # Append text before the entity
        highlighted_text += "\033[1;33m" + text[start:end] + "\033[0m"  # Highlight the entity
        current_pos = end  # Update current position

    highlighted_text += text[current_pos:]  # Append remaining text
    return highlighted_text

# Load data from JSON file
with open("C:\\Users\\HShah\\OneDrive - Genzeon Corporation\\Desktop\\ner_dataset\\vitals\\generated_data.json", "r") as file:
    data = json.load(file)

# Iterate over each element in the JSON
for entry in data:
    if isinstance(entry, list) and len(entry) == 2:
        text = entry[0]
        entities = entry[1]["entities"]
        highlighted_text = highlight_text(text, entities)
        print(highlighted_text)
    else:
        print("Invalid JSON format for entry:", entry)


Patient: Ling Chi, Gender: Male, Admit Date: 10/02/22, VITALS: TEMP. 102.6, PULSE: 47, RESP: 17
Name: John Brown Account No: 12345 Attending Physician: JeffT. Moore, M.D. Consulting Physician Adm Date: 11/12/1999 DOB: 09/10/44 Community General Hospital Anytown, USA PATIENT ADMISSION ASSESSMENT Date 1/IN44 Time 1600 Admit from: Dr. Office ERSHome Other Health Care Facility:. Reason for Adrission: (Patient's own words). Hechia Surgry Primary Care Physician:. 1i Mloore Previous Health Problems (Patient Only) NA 1. Diabetes 2. Epilepsy/seizure disorder 3. High Blood Pressure 4. Heart Disease 5. Kidney Disease 6. Cancer Medications Anesthetics Other. Comments by nymber and other hosptalationsurgenes Dreyibus lnia repr 7. Respiratory 8. GU/GYN 9. GI 10. Steroid use D 11. Flu Vaccine Current CP N 12. Pneumonia Vaccine current Y(Nnog give patient information VITALS: TEMP. 97.9 PULSE 71 RESP 22 B/P 142/61 mmHg Wt 19096s ALLERGIES HABITS NVA Food Environmental Dyes Ruber/atexbalons: Yes (NONA Y

METHOD TO GENERATE 'N' SAMPLES


In [31]:
def generate_training_data(placeholder_to_generator, templates, num_samples):
    training_data = []

    for _ in range(num_samples):
        template = random.choice(templates)
        updated_text, entities = replace_placeholders(
            template, placeholder_to_generator
        )
        training_data.append((updated_text, entities))

    return training_data

In [40]:
# Example usage
num_samples = 5
training_data = generate_training_data(placeholder_to_generator, templates, num_samples)

for idx, sample in enumerate(training_data[::-1]):
    print(f"Sample {idx}\nText:{sample[0]}\nEntities:{sample[1]['entities']}")

Sample 0
Text:Vitals - Temp: 87.7, Pulse: 99, Resp: 20, BP: 84/74
Entities:[(15, 19, 'VITAL-TEMP'), (28, 30, 'VITAL-PULSE'), (38, 40, 'VITAL-RESP'), (46, 51, 'VITAL-BP')]
Sample 1
Text:T - 91.2, HR - 72, BP - 91/89mmHg, RR - 14/min
Entities:[(4, 8, 'VITAL-TEMP'), (15, 17, 'VITAL-PULSE'), (24, 29, 'VITAL-BP'), (40, 42, 'VITAL-RESP')]
Sample 2
Text:Patient: Maria Gates, Gender: F, Admit Date: 07/22/2021, VITALS: TEMP. 86.0, RESP: 25
Entities:[(71, 75, 'VITAL-TEMP'), (83, 85, 'VITAL-RESP')]
Sample 3
Text:Name: Sam Wood, DOB: 01/01/99, Admit Date: 2/27/20, VITALS: TEMP. 83.0, HR: 106
Entities:[(66, 70, 'VITAL-TEMP'), (76, 79, 'VITAL-PULSE')]
Sample 4
Text:Patient's Vitals: Temp - 82.8, Pulse - 56, Resp - 26, BP - 124/79 | Patient: Lisa Banks
Entities:[(25, 29, 'VITAL-TEMP'), (39, 41, 'VITAL-PULSE'), (50, 52, 'VITAL-RESP'), (59, 65, 'VITAL-BP')]


In [41]:
#Generating 10000 records
training_data = generate_training_data(placeholder_to_generator, templates, num_samples=10000)

SAVING TO A PICKLE FILE AND JSON FILE


In [50]:
import json

with open("C:\\Users\\HShah\\OneDrive - Genzeon Corporation\\Desktop\\ner_dataset\\vitals\\training_data\\training_data.json", "w") as file:
    json.dump(training_data, file, indent=4)

In [52]:
#Checking start and end for training data

#Highlighting text to see if spans are correct

import json

def highlight_text(text, entities):
    highlighted_text = ""
    current_pos = 0

    for entity in entities:
        start, end, label = entity
        highlighted_text += text[current_pos:start]  # Append text before the entity
        highlighted_text += "\033[1;33m" + text[start:end] + "\033[0m"  # Highlight the entity
        current_pos = end  # Update current position

    highlighted_text += text[current_pos:]  # Append remaining text
    return highlighted_text

# Load data from JSON file
with open("C:\\Users\\HShah\\OneDrive - Genzeon Corporation\\Desktop\\ner_dataset\\vitals\\training_data\\training_data.json", "r") as file:
    data = json.load(file)

# Iterate over each element in the JSON
for entry in data[:50]:
    if isinstance(entry, list) and len(entry) == 2:
        text = entry[0]
        entities = entry[1]["entities"]
        highlighted_text = highlight_text(text, entities)
        print(highlighted_text)
    else:
        print("Invalid JSON format for entry:", entry)


Patient: Lionel Ronaldo, Admit Date: 09/11/1991, Hospital: Madrid Hospital for Children, VITALS: RESP 28, TEMP. 95.8
Abnormal Vitals Recorded: Temp - 84.7°F (High), Pulse - 85bpm (Low), BP - 126/60 (High), Resp - 28/min (Low)
81.5F, 83bpm, 22/min, BP: 149/80mmHg
Name: Sam Wood, DOB: 01/01/99, Admit Date: 2/27/20, VITALS: TEMP. 99.3, HR: 82
Name: Issac Ferrari, ID: 22928, Hospital: Greenbull Hosptial, VITALS: TEMP. 103.5, PULSE: 40, B/P 90/80
Patient's Vitals: Temp - 101.1, Pulse - 91, Resp - 25, BP - 130/87 | Patient: Lisa Banks
Name: Karan Gupta, ID: 792424, DOB: 7/9/24, VITALS: TEMP. 104.6, HEART RATE: 104, B/P 150/71
Vital Readings: T - 81.9°F, HR - 131bpm, BP - 130/77, RR - 28/minVITALS: Temp - 82.4, Pulse - 112, Resp - 11, BP - 98/88
92.8°F, 117 bpm, 8 breaths/min, BP: 81/73
Patient's Vitals: Temp - 82.6, Pulse - 132, Resp - 20, BP - 130/67 | Patient: Lisa Banks
Name: Karan Gupta, ID: 792424, DOB: 7/9/24, VITALS: TEMP. 91.0, HEART RATE: 49, B/P 83/81
95.8°F, 120 bpm, 23 breaths/mi

In [47]:
import os
import pickle

pickle_file_path = "C:\\Users\\HShah\\OneDrive - Genzeon Corporation\\Desktop\\ner_dataset\\vitals\\training_data\\training_data.pkl"

with open(pickle_file_path, "wb") as file:
    pickle.dump(training_data, file)